# This notebook is for my learning.
I have implemented the followings in pytorch.
* Layer-Wise Learning Rate Dacay
* Attention pooling
* WeightedLayerPooling
* Fast Gradient Method
* Adversarial Weight Perturbation
* Re-initializing upper layer (normal, xavier_uniform, xavier_normal, kaiming_uniform, kaiming_normal, orthogonal)
* Initializing module (normal, xavier_uniform, xavier_normal, kaiming_uniform, kaiming_normal, orthogonal)
* Freeze lower layer when you use very large model (v2-xlarge, funnnel, etc.)
* Loss function, SmoothL1 or RMSE


I will continue to update it if time allows.  
And if you like this notebook, please upvote it. Thank you in advance

If you have time, please check my other notebooks.

* Inference : https://www.kaggle.com/kojimar/fb3-single-pytorch-model-inference
* Ensemble : https://www.kaggle.com/code/kojimar/fb3-deberta-family-inference-9-28-updated

In [ ]:
import datetime
print("開始時間： ", datetime.datetime.now() + datetime.timedelta(hours=9))

開始時間：  2022-11-22 20:48:16.400736


In [ ]:
!nvidia-smi

Tue Nov 22 11:48:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import sys
from pathlib import Path
import os
import json

if 'google.colab' in sys.modules:  # colab環境
    # INPUT = Path(‘/content/input/’)
    print('google colaboratory環境です！')
    # if not os.path.exists('/content/train.csv'):
    if not os.path.exists('../input/feedback-prize-english-language-learning/train.csv'):
        !pip install kaggle
        from google.colab import drive
        drive.mount('/content/drive')

        f = open("/content/drive/MyDrive/kaggle.json", 'r')
        json_data = json.load(f) 
        os.environ['KAGGLE_USERNAME'] = json_data['username']
        os.environ['KAGGLE_KEY'] = json_data['key']

        !kaggle competitions download -c feedback-prize-english-language-learning -p '../input'
        # !kaggle datasets download -d yasufuminakama/fb3-pip-wheels -p '/content/datasets'
        # !unzip '/content/feedback-prize-english-language-learning.zip'
        # !unzip '../input/feedback-prize-english-language-learning.zip'
        !unzip -o '../input/feedback-prize-english-language-learning.zip' -d '../input/feedback-prize-english-language-learning' # -oオプションですでに解凍済みファイルがあっても強制的に上書きする。-dで展開先ディレクトリを指定。

        !pip install transformers==4.21.2
        !pip install tokenizers==0.12.1
        !pip install sentencepiece

        # # 使用するデータセットをダウンロード
        # if not os.path.exists('/content/datasets'):

        #     !kaggle datasets download -d kojimar/fb3models -p '/content/datasets' --unzip
        #     !kaggle datasets download -d randyli2/bert-shopping-mall -p '/content/datasets' --unzip
        #     !kaggle datasets download -d hiromoon166/deberta-v3-large -p '/content/datasets' --unzip
        #     !kaggle datasets download -d tabchenjlu/facebook-bart-large-mnli -p '/content/datasets' --unzip
        #     !kaggle datasets download -d quangphm/fb3embeddings -p '/content/datasets' --unzip
        #     !kaggle datasets download -d tdjogi010/hugging-face-gpt2 -p '/content/datasets' --unzip
        #     !kaggle datasets download -d sauravmaheshkar/huggingface-deberta-variants -p '/content/datasets' --unzip
        #     !kaggle datasets download -d lucca9211/iterativestratification -p '/content/datasets' --unzip
        #     !kaggle datasets download -d yzdivergence/muppet-roberta-large -p '/content/datasets' --unzip
        #     !kaggle datasets download -d yasufuminakama/fb3-pip-wheels -p '/content/datasets' --unzip

    # このノートブック名を取得
    from requests import get
    name_notebook = get('http://172.28.0.2:9000/api/sessions').json()[0]['name']

elif 'kaggle_web_client' in sys.modules:  # kaggle環境
    print('kaggle環境です！')
    # INPUT = Path(‘../input/’)
    pass




google colaboratory環境です！
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive
  0% 0.00/2.80M [00:00<?, ?B/s]
100% 2.80M/2.80M [00:00<00:00, 76.5MB/s]
Archive:  ../input/feedback-prize-english-language-learning.zip
  inflating: ../input/feedback-prize-english-language-learning/sample_submission.csv  
  inflating: ../input/feedback-prize-english-language-learning/test.csv  
  inflating: ../input/feedback-prize-english-language-learning/train.csv  
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.2 MB/s 
     |████████████████████████████████| 6.6 MB 40.7 MB/s 
     |████████████████████████████████| 182 kB 66.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |█

In [ ]:
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import datetime
import math
import string
import pickle
import random
import joblib
import itertools
from distutils.util import strtobool
import warnings
warnings.filterwarnings('ignore')

import scipy as sp
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
from torch.utils.checkpoint import checkpoint

from pdb import set_trace as st

import transformers
import tokenizers
print(f'transformers.__version__: {transformers.__version__}')
print(f'tokenizers.__version__: {tokenizers.__version__}')
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
os.environ['TOKENIZERS_PARALLELISM']='true'

transformers.__version__: 4.21.2
tokenizers.__version__: 0.12.1


# Config

In [ ]:
# ver_title = "AttentionPool(GERU) FGM=True"
# ver_title = "AttPool(Tanh) FGM=False"
# ver_title = "AtPool(Tanh) FGM maxlen=1024"
# ver_title = "AtPool(Tanh) FGM reinit_n=2" # maxlen=512
# ver_title = "AtPool(Tanh) FGM xavier_uni" # maxlen=512
# ver_title = "AtPool(Tanh) FGM pretrain=False" # maxlen=512
# ver_title = "APo(Th)FGM 768　kaiming_uniform"
# ver_title = "APo(Th)FGM 768　kaiming_normal"
# ver_title = "APo(Th)FGM 768　normal 10(scaled)"
# ver_title = "APo(Th) 768　normal lrd85"
# ver_title = "APo(Th) 768　normal lrd95"
# ver_title = "APo(Th) normal maxlen=None"
ver_title = "APo(Th) weight=orthogonal"

In [ ]:
class CFG:
    str_now = (datetime.datetime.now() + datetime.timedelta(hours=9)).strftime('%Y%m%d-%H%M%S')
    train = True
    debug = False
    offline = False
    models_path = 'FB3-models'
    epochs = 5
    # epochs = 4
    save_all_models = False
    competition = 'FB3'
    apex = True
    print_freq = 20
    num_workers = 4
    model = 'microsoft/deberta-v3-base' #If you want to train on the kaggle platform, v3-base is realistic. v3-large will time out.
    loss_func = 'SmoothL1' # 'SmoothL1', 'RMSE'
    gradient_checkpointing = True
    scheduler = 'cosine'
    batch_scheduler = True
    num_cycles = 0.5
    num_warmup_steps = 0
    encoder_lr = 2e-5
    decoder_lr = 2e-5
    min_lr = 1e-6
    #Layer-Wise Learning Rate Decay
    llrd = True
    layerwise_lr = 5e-5
    # layerwise_lr = 5e-8 # 変更している
    layerwise_lr_decay = 0.9
    # layerwise_lr_decay = 0.85 # 変更している
    # layerwise_lr_decay = 0.95 # 変更している
    layerwise_weight_decay = 0.01
    layerwise_adam_epsilon = 1e-6
    layerwise_use_bertadam = False
    #pooling
    # TODO: weightedlayerを試す！！！！！！！！
    # pooling = 'mean' # mean, max, min, attention, weightedlayer
    # pooling = 'weightedlayer'
    pooling = 'attention' # mean, max, min, attention, weightedlayer
    # TODO: 9とか12とかも試してみる
    layer_start = 4
    # layer_start = 9
    # fgm = False
    fgm = True
    awp = False
    # awp = True
    # TODO: 変更してみる！！！
    # こんにちは。特に1000でなければならない理由はありません。私が実験した限りでは、ノルムを小さくするとCVは向上しますが、LBは悪化します。
    max_grad_norm = 1000
    # max_grad_norm = 6554
    # max_grad_norm = 10
    unscale = False
    # unscale = True
    #init_weight
    # init_weight = 'normal' # normal, xavier_uniform, xavier_normal, kaiming_uniform, kaiming_normal, orthogonal
    init_weight = 'orthogonal'
    # use_pretrained = False
    use_pretrained = True
    #re-init
    reinit = True
    # TODO: 2層をreinitializeしてみる
    reinit_n = 1
    # reinit_n = 2
    #adversarial
    adv_lr = 1
    adv_eps = 0.2
    eps = 1e-6
    betas = (0.9, 0.999)
    # max_len = 512
    max_len = 768
    # max_len = 1024
    weight_decay = 0.01
    gradient_accumulation_steps = 1
    target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed = 42
    cv_seed = 42
    # seed = 2022
    # cv_seed = 2022
    # TODO: 10foldの方が良いのかも...
    n_fold = 4
    trn_fold = list(range(n_fold))
    batch_size = 8
    n_targets = 6
    gpu_id = 0
    device = f'cuda:{gpu_id}'
    train_file = '../input/feedback-prize-english-language-learning/train.csv'
    test_file = '../input/feedback-prize-english-language-learning/test.csv'
    submission_file = '../input/feedback-prize-english-language-learning/sample_submission.csv'

In [ ]:
#Unique model name
if len(CFG.model.split("/")) == 2:
    CFG.identifier = f'{CFG.str_now}-{CFG.model.split("/")[1]}'
else:
    CFG.identifier = f'{CFG.str_now}-{CFG.model}'

# CFG.identifier = 'made-models-of-deberta-v3-base'
    
print(CFG.identifier)

20221122-204903-deberta-v3-base


In [ ]:
# ################################
# nunique_tokenの実装
# ################################
# def get_text_token_cnts(
#     df: pd.DataFrame,
#     tokenizer,
#     inplace: bool = False
# ):
#     token_cnts = []
#     for text in tqdm(df["full_text"], total=len(df)):
#         input_ids = tokenizer(
#             text,
#             add_special_tokens=False,
#             return_token_type_ids=False,
#             return_attention_mask=False
#         )["input_ids"]
#         token_cnts.append([len(input_ids), len(set(input_ids))])
#     if not inplace:
#         df = df.copy()
#     df[["n_token", "nunique_token"]] = token_cnts
#     return df

# # train_df = pd.read_csv(f"{CFG.data_dir}/train.csv")


In [ ]:
if CFG.train:
    CFG.df_train = pd.read_csv(CFG.train_file)

    #################################
    # nunique_tokenの実装
    #################################
    # CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.model)
    # CFG.tokenizer.save_pretrained(CFG.OUTPUT_DIR + 'tokenizer')

    # train_df = get_text_token_cnts(CFG.df_train, CFG.tokenizer, inplace=False)
    # N_TOKEN_BIN = 50
    # N_UNIQUE_TOKEN_BIN = 50
    # train_df["n_token_bin"] = pd.qcut(
    #     x=train_df["n_token"], q=N_TOKEN_BIN, labels=False, duplicates="drop"
    # )
    # train_df["nutoken_bin"] = pd.qcut(
    #     x=train_df["nunique_token"], q=N_UNIQUE_TOKEN_BIN, labels=False, duplicates="drop"
    # )


    CFG.OUTPUT_DIR = f'./{CFG.identifier}/'
    CFG.log_filename = CFG.OUTPUT_DIR + 'train'
    if CFG.offline:
        #TO DO
        pass
    else:
        os.system('pip install iterative-stratification==0.1.7')
    #CV
    from iterstrat.ml_stratifiers import MultilabelStratifiedKFold    
    Fold = MultilabelStratifiedKFold(n_splits = CFG.n_fold, shuffle = True, random_state = CFG.cv_seed)

    for n, (train_index, val_index) in enumerate(Fold.split(CFG.df_train, CFG.df_train[CFG.target_cols])):
    # for n, (train_index, val_index) in enumerate(Fold.split(CFG.df_train, CFG.df_train[CFG.target_cols + ["n_token_bin", "nutoken_bin"]])):


        CFG.df_train.loc[val_index, 'fold'] = int(n)
    CFG.df_train['fold'] = CFG.df_train['fold'].astype(int)
else:
    #TO DO
    pass

if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]
    if CFG.train:
        CFG.df_train = CFG.df_train.sample(n = 100, random_state = CFG.seed).reset_index(drop=True)
        
os.makedirs(CFG.OUTPUT_DIR, exist_ok=True)    
print(CFG.OUTPUT_DIR)

./20221122-204903-deberta-v3-base/


In [ ]:
from pathlib import Path

# %cdで/rootに移動できる。/root/.kaggle にkaggle.jsonを配置しなくてはいけないっぽいから、/root/.kaggleディレクトリ作って、そこにkaggle.jsonを配置して、その後もとの/contentに戻っている
%cd
!mkdir .kaggle
!cp /content/drive/MyDrive/kaggle.json .kaggle/
%cd /content

from kaggle.api.kaggle_api_extended import KaggleApi

# 環境変数設定
%env KAGGLE_USERNAME=""

PRETRAIN_DIR = Path(CFG.OUTPUT_DIR)
api = KaggleApi()
api.authenticate()
# dataset-metadata.jsonを作成
api.dataset_initialize_cli(folder=PRETRAIN_DIR)

# Helper function

In [ ]:
def MCRMSE(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:, i]
        y_pred = y_preds[:, i]
        score = mean_squared_error(y_true, y_pred, squared = False)
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores

def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores

def get_logger(filename = CFG.log_filename):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter('%(message)s'))
    handler2 = FileHandler(filename = f'{filename}.log')
    handler2.setFormatter(Formatter('%(message)s'))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

def prepare_input(cfg, text):
    inputs = cfg.tokenizer.encode_plus(
        text,
        return_tensors = None,
        add_special_tokens = True,
        max_length = cfg.max_len,
        pad_to_max_length = True,
        truncation = True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype = torch.long)
    return inputs



# def prepare_input(cfg, text):
#     inputs = cfg.tokenizer.encode_plus(
#         text,
#         return_tensors=None,
#         return_token_type_ids=False,
#         add_special_tokens=True,
#         max_length=None,
#         truncation=False,

#     )
#     for k, v in inputs.items():
#         inputs[k] = torch.tensor(v, dtype=torch.long)
#     return inputs  



def collate(inputs):
    mask_len = int(inputs['attention_mask'].sum(axis = 1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:, :mask_len]
    return inputs

class AverageMeter(object):
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        
    def update(self, val, n = 1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return f'{int(m)}m {int(s)}s'

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return f'{str(asMinutes(s))} (remain {str(asMinutes(rs))})'

def seed_everything(seed = 42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
class RMSELoss(nn.Module):
    def __init__(self, reduction = 'mean', eps = 1e-9):
        super().__init__()
        self.mse = nn.MSELoss(reduction = 'none')
        self.reduction = reduction
        self.eps = eps
        
    def forward(self, y_pred, y_true):
        loss = torch.sqrt(self.mse(y_pred, y_true) + self.eps)
        if self.reduction == 'none':
            loss = loss
        elif self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()
        return loss
    
seed_everything(CFG.seed)

# Pooling

* Attention pooling (https://www.kaggle.com/competitions/feedback-prize-english-language-learning/discussion/361678)
* WeightedLayerPooling (https://www.kaggle.com/code/rhtsingh/on-stability-of-few-sample-transformer-fine-tuning?scriptVersionId=67176591&cellId=19)

In [ ]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min = 1e-9)
        mean_embeddings = sum_embeddings/sum_mask
        return mean_embeddings

class MaxPooling(nn.Module):
    def __init__(self):
        super(MaxPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        embeddings = last_hidden_state.clone()
        embeddings[input_mask_expanded == 0] = -1e4
        max_embeddings, _ = torch.max(embeddings, dim = 1)
        return max_embeddings
    
class MinPooling(nn.Module):
    def __init__(self):
        super(MinPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        embeddings = last_hidden_state.clone()
        embeddings[input_mask_expanded == 0] = 1e-4
        min_embeddings, _ = torch.min(embeddings, dim = 1)
        return min_embeddings

#Attention pooling
class AttentionPooling(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.attention = nn.Sequential(
        nn.Linear(in_dim, in_dim),
        nn.LayerNorm(in_dim),
        # nn.GELU(),
        nn.Tanh(),
        nn.Linear(in_dim, 1),
        )

    def forward(self, last_hidden_state, attention_mask):
        w = self.attention(last_hidden_state).float()
        w[attention_mask==0]=float('-inf')
        w = torch.softmax(w,1)
        attention_embeddings = torch.sum(w * last_hidden_state, dim=1)
        return attention_embeddings

#There may be a bug in my implementation because it does not work well.
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        # num_hidden_layers = 12
        self.num_hidden_layers = num_hidden_layers
        # 埋め込み層の出力も受け取っているため、モデルでは隠れ層が12層あるにもかかわらず、隠れ層の出力は13層になっています。
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, ft_all_layers):
        all_layer_embedding = torch.stack(ft_all_layers)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :] # n層目以降のembedding層だけ

        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        # 特定の重み（weight_factor）をlayer_startで指定した層より後の層全てに掛け合わせて、そしてlayer_weightsを合計で割って平均を算出する。
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average



# Fast Gradient Method (FGM)
Reference :

https://www.kaggle.com/c/tweet-sentiment-extraction/discussion/143764

https://blog.brainpad.co.jp/entry/2022/08/23/153001#:~:text=FGM%EF%BC%88Fast%20Gradient%20Method%EF%BC%89,-FGM%E3%81%A7%E3%81%AF%E3%80%81%E5%AD%A6%E7%BF%92&text=%E5%AD%A6%E7%BF%92%E3%81%AE%E5%90%84step%E3%81%8A,%E3%81%93%E3%81%A8%E3%81%8C%E5%8F%AF%E8%83%BD%E3%81%A8%E3%81%AA%E3%82%8A%E3%81%BE%E3%81%99%E3%80%82

In [ ]:
class FGM():
    def __init__(self, model):
        self.model = model
        self.backup = {}

    def attack(self, epsilon = 1., emb_name = 'word_embeddings'):
        """
        敵対的な摂動(小さな攪乱(かくらん)・ずれ。)を求め、現在のembedding layerに摂動を加える
        """
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0:
                    r_at = epsilon * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name = 'word_embeddings'):
        """
        敵対的な摂動を求める際に変更してしまったembedding layerのパラメータについて
        元のパラメータを代入する
        """
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                assert name in self.backup
                param.data = self.backup[name]
            self.backup = {}


# Adversarial Weight Perturbation (AWP)
There may be a bug in my implementation because it does not work well.

Reference : 

https://www.kaggle.com/code/wht1996/feedback-nn-train/notebook

In [ ]:
# FGMにおいては、敵対的な摂動はモデルの入力（入力の埋め込み表現）に対して加えられることになりますが、摂動をモデルの「入力」ではなくモデルの「重み」に加える、という方法も考えられます。
# この方法として代表的なのがAWP（Adversarial Weight Perturbation）です。
class AWP:
    def __init__(
        self,
        model,
        optimizer,
        adv_param="weight",
        adv_lr=1,
        adv_eps=0.2,
        start_epoch=0,
        adv_step=1,
        scaler=None
    ):
        self.model = model
        self.optimizer = optimizer
        self.adv_param = adv_param
        self.adv_lr = adv_lr
        self.adv_eps = adv_eps
        self.start_epoch = start_epoch
        self.adv_step = adv_step
        self.backup = {}
        self.backup_eps = {}
        self.scaler = scaler
        
    # def attack_backward(self, x, y, attention_mask,epoch):
    #     """
    #     敵対的な摂動を加えた損失を計算し、パラメータを更新する
    #     """
    #     if (self.adv_lr == 0) or (epoch < self.start_epoch):
    #         return None

    #     self._save() 
    #     for i in range(self.adv_step):
    #         self._attack_step() 
    #         with torch.cuda.amp.autocast():
    #             adv_loss, tr_logits = self.model(input_ids=x, attention_mask=attention_mask, labels=y)
    #             adv_loss = adv_loss.mean()
    #         self.optimizer.zero_grad()
    #         self.scaler.scale(adv_loss).backward()
    #     self._restore()

    # see: https://www.kaggle.com/code/kojimar/fb3-single-pytorch-model-train/comments#2017711
    def attack_backward(self, x, y ,epoch):
        """
        敵対的な摂動を加えた損失を計算し、パラメータを更新する
        """
        if (self.adv_lr == 0) or (epoch < self.start_epoch):
            return None
        self._save() 
        for i in range(self.adv_step):
            self._attack_step() 
            with torch.cuda.amp.autocast():
                y_preds = self.model(x)
                adv_loss = self.criterion(y_preds,y)
                adv_loss = adv_loss.mean()
            self.optimizer.zero_grad()
            self.scaler.scale(adv_loss).backward()
        self._restore()

        
    def _attack_step(self):
        """
        敵対的な摂動を求め、重みに加える
        重みの範囲をbackup_epsで制限している
        """
        e = 1e-6
        for name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None and self.adv_param in name:
                norm1 = torch.norm(param.grad)
                norm2 = torch.norm(param.data.detach())
                if norm1 != 0 and not torch.isnan(norm1):
                    r_at = self.adv_lr * param.grad / (norm1 + e) * (norm2 + e)
                    param.data.add_(r_at)
                    param.data = torch.min(
                        torch.max(param.data, self.backup_eps[name][0]), self.backup_eps[name][1]
                    )
                # param.data.clamp_(*self.backup_eps[name])

    def _save(self):
        """
        重みのバックアップと、重みの範囲を取得する
        重みの範囲はパラメータの絶対値とadv_epsによって決定する
        """
        for name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None and self.adv_param in name:
                if name not in self.backup:
                    self.backup[name] = param.data.clone()
                    grad_eps = self.adv_eps * param.abs().detach()
                    self.backup_eps[name] = (
                        self.backup[name] - grad_eps,
                        self.backup[name] + grad_eps,
                    )

    def _restore(self,):
        """
        バックアップを取っていたパラメータを代入するとともに初期化する
        """
        for name, param in self.model.named_parameters():
            if name in self.backup:
                param.data = self.backup[name]
        self.backup = {}
        self.backup_eps = {}

# Train function
* FGM
* AWP (There may be a bug.)
* Unscale optimizer

In [ ]:
def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    losses = AverageMeter()
    model.train()
    # init_scale=4096 参考：https://qiita.com/bowdbeg/items/71c62cf8ef891d164ecd#%E6%B3%A8%E6%84%8F%E7%82%B9
    # scaler = torch.cuda.amp.GradScaler(enabled = CFG.apex, init_scale=4096)
    scaler = torch.cuda.amp.GradScaler(enabled = CFG.apex)
    start = end = time.time()
    global_step = 0
    if CFG.fgm:
        # embedding layerに敵対的な摂動を加える
        fgm = FGM(model)
    if CFG.awp:
        awp = AWP(model,
                  optimizer, 
                  adv_lr = CFG.adv_lr, 
                  adv_eps = CFG.adv_eps, 
                  scaler = scaler)
    for step, (inputs, labels) in enumerate(train_loader):
        attention_mask = inputs['attention_mask'].to(device)
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.cuda.amp.autocast(enabled = CFG.apex):
            y_preds = model(inputs)
            # 敵対的な摂動を加えられた状態での損失を計算
            loss = criterion(y_preds, labels)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        if CFG.unscale:
            scaler.unscale_(optimizer)
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        
        #Fast Gradient Method (FGM)
        if CFG.fgm:
            fgm.attack()
            with torch.cuda.amp.autocast(enabled = CFG.apex):
                y_preds = model(inputs)
                loss_adv = criterion(y_preds, labels)
                loss_adv.backward()
            fgm.restore()
            
        #Adversarial Weight Perturbation (AWP)
        if CFG.awp:
            loss_awp = awp.attack_backward(inputs, labels, step + 1)
            # loss_awp = awp.attack_backward(inputs, labels, attention_mask, step + 1)
            loss_awp.backward()
            awp._restore()
        

        # if (step + 1) % CFG.gradient_accumulation_steps == 0:
        #     # 超参考： https://qiita.com/bowdbeg/items/71c62cf8ef891d164ecd#%E6%B3%A8%E6%84%8F%E7%82%B9
        #     # unscale_ -> 勾配を直接操作する場合には，勾配の大きさを一度戻してから操作をします．例では勾配のクリッピング（勾配の最大値を決めておいて，超えたら小さくする）を挙げていますが，勾配が大きいままクリッピングすると間違った結果（おそらくすべてクリッピングされる）が出てしまいます．
        #     scaler.unscale_(optimizer) # クリップ時に正しくできるように一度スケールを戻す
        #     grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        #     scaler.step(optimizer) # パラメタの更新
        #     scaler.update() # スケールの更新
        #     optimizer.zero_grad()
        #     global_step += 1
        #     if CFG.batch_scheduler:
        #         scheduler.step()

        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()

        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader) - 1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f} '
                  'LR: {lr:.8f} '
                  .format(epoch + 1, step, len(train_loader), remain = timeSince(start, float(step + 1)/len(train_loader)),
                          loss = losses,
                          grad_norm = grad_norm,
                          lr = scheduler.get_lr()[0]
                         )
                 )
    return losses.avg

# Valid function

In [ ]:
def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(inputs)
            loss = criterion(y_preds, labels)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader) - 1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss = losses,
                          remain = timeSince(start, float(step + 1) / len(valid_loader))
                         )
                 )
    return losses.avg, np.concatenate(preds)

# Logger

In [ ]:
LOGGER = get_logger()
LOGGER.info(f'OUTPUT_DIR: {CFG.OUTPUT_DIR}')

OUTPUT_DIR: ./20221122-204903-deberta-v3-base/
INFO:__main__:OUTPUT_DIR: ./20221122-204903-deberta-v3-base/


# Tokenizer

In [ ]:
CFG.tokenizer = AutoTokenizer.from_pretrained(CFG.model)
CFG.tokenizer.save_pretrained(CFG.OUTPUT_DIR + 'tokenizer')

#max_len
lengths = []
tk0 = tqdm(CFG.df_train['full_text'].fillna('').values, total = len(CFG.df_train))
for text in tk0:
    length = len(CFG.tokenizer(text, add_special_tokens = False)['input_ids'])
    lengths.append(length)
CFG.max_len = max(lengths) + 2
LOGGER.info(f'max_len: {CFG.max_len}')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/3911 [00:00<?, ?it/s]

max_len: 1428
INFO:__main__:max_len: 1428


# Dataset

In [ ]:
class FB3TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['full_text'].values
        self.labels = df[cfg.target_cols].values
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        label = torch.tensor(self.labels[item], dtype = torch.float)
        return inputs, label

# Model

* Initializing module (normal, xavier_uniform, xavier_normal, kaiming_uniform, kaiming_normal, orthogonal) 
* Freeze lower layer when you use large model (v2-xlarge, funnnel, etc.)

モジュールの初期化 (normal、xavier_uniform、xavier_normal、kaiming_uniform、kaiming_normal、orthogonal)  
大きなモデル（v2-xlarge, funnnelなど）を使用する場合、下のレイヤーをフリーズさせる。

In [ ]:
class FB3Model(nn.Module):
    def __init__(self, CFG, config_path = None, pretrained = False):
        super().__init__()
        self.CFG = CFG
        if config_path is None:
            self.config = AutoConfig.from_pretrained(CFG.model, output_hidden_states = True)
            self.config.save_pretrained(CFG.OUTPUT_DIR + 'config')
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
        else:
            self.config = torch.load(config_path)
            
        LOGGER.info(self.config)
        
        if pretrained:
            self.model = AutoModel.from_pretrained(CFG.model, config=self.config)
            self.model.save_pretrained(CFG.OUTPUT_DIR + 'model')
        else:
            #self.model = AutoModel(self.config)<- this is wrong
            self.model = AutoModel.from_config(self.config)#<- this is correct


        if self.CFG.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()
            
        if CFG.pooling == 'mean':
            self.pool = MeanPooling()
        elif CFG.pooling == 'max':
            self.pool = MaxPooling()
        elif CFG.pooling == 'min':
            self.pool = MinPooling()
        elif CFG.pooling == 'attention':
            self.pool = AttentionPooling(self.config.hidden_size)
        elif CFG.pooling == 'weightedlayer':
            self.pool = WeightedLayerPooling(self.config.num_hidden_layers, layer_start = CFG.layer_start, layer_weights = None)        
        
        # self.config.hidden_size = 768
        # self.CFG.n_targets(6)
        # 768個の最終的なベクトル値から、６つの出力値を算出してほしい。
        self.fc = nn.Linear(self.config.hidden_size, self.CFG.n_targets)
        self._init_weights(self.fc)
        
        if 'deberta-v2-xxlarge' in CFG.model:
            self.model.embeddings.requires_grad_(False)
            self.model.encoder.layer[:24].requires_grad_(False)
        if 'deberta-v2-xlarge' in CFG.model:
            self.model.embeddings.requires_grad_(False)
            self.model.encoder.layer[:12].requires_grad_(False)
        if 'funnel-transformer-xlarge' in CFG.model:
            self.model.embeddings.requires_grad_(False)
            self.model.encoder.blocks[:1].requires_grad_(False)
        if 'funnel-transformer-large' in CFG.model:
            self.model.embeddings.requires_grad_(False)
            self.model.encoder.blocks[:1].requires_grad_(False)
        if 'deberta-large' in CFG.model:
            self.model.embeddings.requires_grad_(False)
            self.model.encoder.layer[:16].requires_grad_(False)
        if 'deberta-xlarge' in CFG.model:
            self.model.embeddings.requires_grad_(False)
            self.model.encoder.layer[:36].requires_grad_(False)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            if CFG.init_weight == 'normal':
                module.weight.data.normal_(mean = 0.0, std = self.config.initializer_range)
            elif CFG.init_weight == 'xavier_uniform':
                module.weight.data = nn.init.xavier_uniform_(module.weight.data)
            elif CFG.init_weight == 'xavier_normal':
                module.weight.data = nn.init.xavier_normal_(module.weight.data)
            elif CFG.init_weight == 'kaiming_uniform':
                module.weight.data = nn.init.kaiming_uniform_(module.weight.data)
            elif CFG.init_weight == 'kaiming_normal':
                module.weight.data = nn.init.kaiming_normal_(module.weight.data)
            elif CFG.init_weight == 'orthogonal':
                module.weight.data = nn.init.orthogonal_(module.weight.data)
                
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            if CFG.init_weight == 'normal':
                module.weight.data.normal_(mean = 0.0, std = self.config.initializer_range)
            elif CFG.init_weight == 'xavier_uniform':
                module.weight.data = nn.init.xavier_uniform_(module.weight.data)
            elif CFG.init_weight == 'xavier_normal':
                module.weight.data = nn.init.xavier_normal_(module.weight.data)
            elif CFG.init_weight == 'kaiming_uniform':
                module.weight.data = nn.init.kaiming_uniform_(module.weight.data)
            elif CFG.init_weight == 'kaiming_normal':
                module.weight.data = nn.init.kaiming_normal_(module.weight.data)
            elif CFG.init_weight == 'orthogonal':
                module.weight.data = nn.init.orthogonal_(module.weight.data)
                
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    
    def feature(self, inputs):
        outputs = self.model(**inputs)

        if CFG.pooling != 'weightedlayer':
            # 最後の層の形(last_hidden_state)
            # (Pdb) outputs[0].size()
            # torch.Size([8, 676, 768])
            last_hidden_states = outputs[0]
            # last_hidden_states = outputs.last_hidden_state
            feature = self.pool(last_hidden_states, inputs['attention_mask'])
        else:
            # 全ての層の形(hidden_states)
            # (Pdb) outputs[1][0].shape
            # torch.Size([8, 676, 768])
            # (Pdb) outputs[1][1].shape
            # torch.Size([8, 676, 768])
            # (Pdb) outputs[1][2].shape
            # torch.Size([8, 676, 768])

            # all_layer_embeddings = outputs[1]
            all_layer_embeddings = outputs.hidden_states
            # all_hidden_states = torch.stack(all_layer_embeddings)

            # feature = self.pool(all_layer_embeddings)
            # Weighted pooling of last n layers.
            feature = self.pool(all_layer_embeddings)[:, 0]  # Bx768
            
        return feature
    
    def forward(self, inputs):
        feature = self.feature(inputs)
        outout = self.fc(feature)
        return outout

# Train
* Re-initializing upper layer (normal, xavier_uniform, xavier_normal, kaiming_uniform, kaiming_normal, orthogonal) 
* Layer-Wise Learning Rate Dacay (https://www.kaggle.com/code/rhtsingh/on-stability-of-few-sample-transformer-fine-tuning?scriptVersionId=67176591&cellId=29)
* Loss function, SmoothL1 or RMSE

In [ ]:
def re_initializing_layer(model, config, layer_num):
    # layer_num デフォルト=1(最後の１層だけreinitilizeする。)
    for module in model.model.encoder.layer[-layer_num:].modules():
        if isinstance(module, nn.Linear):
            if CFG.init_weight == 'normal':
                module.weight.data.normal_(mean=0.0, std=config.initializer_range)
            elif CFG.init_weight == 'xavier_uniform':
                module.weight.data = nn.init.xavier_uniform_(module.weight.data)
            elif CFG.init_weight == 'xavier_normal':
                module.weight.data = nn.init.xavier_normal_(module.weight.data)
            elif CFG.init_weight == 'kaiming_uniform':
                module.weight.data = nn.init.kaiming_uniform_(module.weight.data)
            elif CFG.init_weight == 'kaiming_normal':
                module.weight.data = nn.init.kaiming_normal_(module.weight.data)
            elif CFG.init_weight == 'orthogonal':
                module.weight.data = nn.init.orthogonal_(module.weight.data) 
                
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            if CFG.init_weight == 'normal':
                module.weight.data.normal_(mean=0.0, std=config.initializer_range)
            elif CFG.init_weight == 'xavier_uniform':
                module.weight.data = nn.init.xavier_uniform_(module.weight.data)
            elif CFG.init_weight == 'xavier_normal':
                module.weight.data = nn.init.xavier_normal_(module.weight.data)
            elif CFG.init_weight == 'kaiming_uniform':
                module.weight.data = nn.init.kaiming_uniform_(module.weight.data)
            elif CFG.init_weight == 'kaiming_normal':
                module.weight.data = nn.init.kaiming_normal_(module.weight.data)
            elif CFG.init_weight == 'orthogonal':
                module.weight.data = nn.init.orthogonal_(module.weight.data)
                
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    return model   

def train_loop(folds, fold):
    LOGGER.info(f"========== fold: {fold} training ==========")
    
    train_folds = folds[folds['fold'] != fold].reset_index(drop = True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop = True)
    valid_labels = valid_folds[CFG.target_cols].values
    
    train_dataset = FB3TrainDataset(CFG, train_folds)
    valid_dataset = FB3TrainDataset(CFG, valid_folds)
    
    train_loader = DataLoader(train_dataset,
                              batch_size = CFG.batch_size,
                              shuffle = True, 
                              num_workers = CFG.num_workers,
                              pin_memory = True, 
                              drop_last = True
                             )
    valid_loader = DataLoader(valid_dataset,
                              batch_size = CFG.batch_size * 2,
                              shuffle=False,
                              num_workers=CFG.num_workers,
                              pin_memory=True, 
                              drop_last=False)


    # from transformers import DataCollatorWithPadding
    # train_loader = DataLoader(train_dataset,
    #                           batch_size = CFG.batch_size,
    #                           shuffle = True, 
    #                           num_workers = CFG.num_workers,
    #                           collate_fn=DataCollatorWithPadding(tokenizer=CFG.tokenizer, padding="longest"),
    #                           pin_memory = True, 
    #                           drop_last = True
    #                          )
    # valid_loader = DataLoader(valid_dataset,
    #                         batch_size = CFG.batch_size * 2,
    #                         shuffle=False,
    #                         num_workers=CFG.num_workers,
    #                         collate_fn=DataCollatorWithPadding(tokenizer=CFG.tokenizer, padding="longest"),
    #                         pin_memory=True, 
    #                         drop_last=False)
    


    model = FB3Model(CFG, config_path = None, pretrained = CFG.use_pretrained)
    # デフォルトTrue
    if CFG.reinit:
        model = re_initializing_layer(model, model.config, CFG.reinit_n) # デフォルト　　reinit_n=１
        
    #os.makedirs(CFG.OUTPUT_DIR + 'config/', exist_ok = True)
    #torch.save(model.config, CFG.OUTPUT_DIR + 'config/config.pth')
    model.to(CFG.device)
    
    def get_optimizer_params(model,
                             encoder_lr,
                             decoder_lr,
                             weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr,
             'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr,
             'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr,
             'weight_decay': 0.0}
        ]
        return optimizer_parameters
    
    #llrd
    def get_optimizer_grouped_parameters(model, 
                                         layerwise_lr,
                                         layerwise_weight_decay,
                                         layerwise_lr_decay):
        
        no_decay = ["bias", "LayerNorm.weight"]
        # initialize lr for task specific layer
        optimizer_grouped_parameters = [{"params": [p for n, p in model.named_parameters() if "model" not in n],
                                         "weight_decay": 0.0,
                                         "lr": layerwise_lr,
                                        },]
        # initialize lrs for every layer
        layers = [model.model.embeddings] + list(model.model.encoder.layer)
        layers.reverse()
        lr = layerwise_lr
        for layer in layers:
            optimizer_grouped_parameters += [{"params": [p for n, p in layer.named_parameters() if not any(nd in n for nd in no_decay)],
                                              "weight_decay": layerwise_weight_decay,
                                              "lr": lr,
                                             },
                                             {"params": [p for n, p in layer.named_parameters() if any(nd in n for nd in no_decay)],
                                              "weight_decay": 0.0,
                                              "lr": lr,
                                             },]
            lr *= layerwise_lr_decay
        return optimizer_grouped_parameters
    
    # デフォルトTrue
    if CFG.llrd:
        from transformers import AdamW
        grouped_optimizer_params = get_optimizer_grouped_parameters(model, 
                                                                    CFG.layerwise_lr, 
                                                                    CFG.layerwise_weight_decay, 
                                                                    CFG.layerwise_lr_decay)
        optimizer = AdamW(grouped_optimizer_params,
                          lr = CFG.layerwise_lr,
                          eps = CFG.layerwise_adam_epsilon,
                          correct_bias = not CFG.layerwise_use_bertadam)
    else:
        from torch.optim import AdamW
        optimizer_parameters = get_optimizer_params(model,
                                                    encoder_lr=CFG.encoder_lr, 
                                                    decoder_lr=CFG.decoder_lr,
                                                    weight_decay=CFG.weight_decay)
        optimizer = AdamW(optimizer_parameters, 
                          lr=CFG.encoder_lr,
                          eps=CFG.eps,
                          betas=CFG.betas)
    
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, 
                num_warmup_steps = cfg.num_warmup_steps, 
                num_training_steps = num_train_steps
            )
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, 
                num_warmup_steps = cfg.num_warmup_steps, 
                num_training_steps = num_train_steps,
                num_cycles = cfg.num_cycles
            )
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)
    
    # デフォルトSmoothL1
    if CFG.loss_func == 'SmoothL1':
        criterion = nn.SmoothL1Loss(reduction='mean')
    elif CFG.loss_func == 'RMSE':
        criterion = RMSELoss(reduction='mean')
    
    best_score = np.inf
    best_train_loss = np.inf
    best_val_loss = np.inf
    
    epoch_list = []
    epoch_avg_loss_list = []
    epoch_avg_val_loss_list = []
    epoch_score_list = []
    epoch_scores_list = []

    for epoch in range(CFG.epochs):
        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, CFG.device)

        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, CFG.device)
        
        # scoring
        score, scores = get_score(valid_labels, predictions)

        elapsed = time.time() - start_time
        
        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}  Scores: {scores}')
        
        epoch_list.append(epoch+1)
        epoch_avg_loss_list.append(avg_loss)
        epoch_avg_val_loss_list.append(avg_val_loss)
        epoch_score_list.append(score)
        epoch_scores_list.append(scores)
        
        if best_score > score:
            best_score = score
            best_train_loss = avg_loss
            best_val_loss = avg_val_loss
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        CFG.OUTPUT_DIR + f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")
            
        # デフォルトFalse(保存するのはベストスコアのモデルのみ)
        if CFG.save_all_models:
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        CFG.OUTPUT_DIR + f"{CFG.model.replace('/', '-')}_fold{fold}_epoch{epoch + 1}.pth")

    predictions = torch.load(CFG.OUTPUT_DIR + f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location = torch.device('cpu'))['predictions']
    valid_folds[[f"pred_{c}" for c in CFG.target_cols]] = predictions
    
    df_epoch = pd.DataFrame({'epoch' : epoch_list,
                             'MCRMSE' : epoch_score_list,
                             'train_loss' : epoch_avg_loss_list, 
                             'val_loss' : epoch_avg_val_loss_list})
    df_scores = pd.DataFrame(epoch_scores_list)
    df_scores.columns = CFG.target_cols
    
    
    torch.cuda.empty_cache()
    gc.collect()
    
    return best_train_loss, best_val_loss, valid_folds, pd.concat([df_epoch, df_scores], axis = 1)

# Run !

In [ ]:
def get_result(oof_df, fold, best_train_loss, best_val_loss):
    labels = oof_df[CFG.target_cols].values
    preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
    score, scores = get_score(labels, preds)
    LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')
    _output_log = pd.DataFrame([CFG.identifier, CFG.model, CFG.cv_seed, CFG.seed, fold, 'best', score, best_train_loss, best_val_loss] + scores).T
    _output_log.columns = ['file', 'model', 'cv_seed', 'seed', 'fold', 'epoch', 'MCRMSE', 'train_loss', 'val_loss'] + CFG.target_cols
    return _output_log

# デフォルトTrue
if CFG.train:
    output_log = pd.DataFrame()
    oof_df = pd.DataFrame()
    train_loss_list = []
    val_loss_list = []
    for fold in range(CFG.n_fold):
        if fold in CFG.trn_fold:
            best_train_loss, best_val_loss, _oof_df, df_epoch_scores = train_loop(CFG.df_train, fold)
            train_loss_list.append(best_train_loss)
            val_loss_list.append(best_val_loss)
            oof_df = pd.concat([oof_df, _oof_df])
            LOGGER.info(f"========== fold: {fold} result ==========")

            df_epoch_scores['file'] = CFG.identifier
            df_epoch_scores['model'] = CFG.model
            df_epoch_scores['cv_seed'] = CFG.cv_seed
            df_epoch_scores['seed'] = CFG.seed
            df_epoch_scores['fold'] = fold
            df_epoch_scores = df_epoch_scores[['file', 'model', 'cv_seed', 'seed', 'fold', 'epoch', 'MCRMSE', 'train_loss', 'val_loss'] + CFG.target_cols]

            _output_log = get_result(_oof_df, fold, best_train_loss, best_val_loss)
            output_log = pd.concat([output_log, df_epoch_scores, _output_log])

    oof_df = oof_df.reset_index(drop=True)
    LOGGER.info(f"========== CV ==========")
    _output_log = get_result(oof_df, 'OOF', np.mean(train_loss_list), np.mean(val_loss_list))
    output_log = pd.concat([output_log, _output_log])
    output_log.to_csv(f'{CFG.identifier}.csv', index=False)
    oof_df.to_pickle(CFG.OUTPUT_DIR+'oof_df.pkl', protocol = 4)

========== fold: 0 training ==========
INFO:__main__:========== fold: 0 training ==========
DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-base",
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.21.2",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

INFO:__main__:Deberta

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: [1][0/366] Elapsed 0m 6s (remain 39m 52s) Loss: 2.8944(2.8944) Grad: inf LR: 0.00005000 
Epoch: [1][20/366] Elapsed 1m 20s (remain 22m 6s) Loss: 0.2239(0.9478) Grad: 249337.7031 LR: 0.00004998 
Epoch: [1][40/366] Elapsed 2m 34s (remain 20m 20s) Loss: 0.0932(0.5955) Grad: 93935.9375 LR: 0.00004994 
Epoch: [1][60/366] Elapsed 3m 35s (remain 17m 58s) Loss: 0.1148(0.4565) Grad: 547389.8750 LR: 0.00004986 
Epoch: [1][80/366] Elapsed 4m 48s (remain 16m 54s) Loss: 0.1707(0.3855) Grad: 408735.4062 LR: 0.00004976 
Epoch: [1][100/366] Elapsed 6m 9s (remain 16m 9s) Loss: 0.1322(0.3432) Grad: 121231.5469 LR: 0.00004963 
Epoch: [1][120/366] Elapsed 7m 29s (remain 15m 10s) Loss: 0.1614(0.3113) Grad: 262559.1562 LR: 0.00004946 
Epoch: [1][140/366] Elapsed 8m 49s (remain 14m 4s) Loss: 0.1220(0.2869) Grad: 155246.5625 LR: 0.00004927 
Epoch: [1][160/366] Elapsed 10m 0s (remain 12m 45s) Loss: 0.1385(0.2662) Grad: 108641.7188 LR: 0.00004905 
Epoch: [1][180/366] Elapsed 11m 3s (remain 11m 17s) Loss:

Epoch 1 - avg_train_loss: 0.1896  avg_val_loss: 0.1172  time: 1528s
INFO:__main__:Epoch 1 - avg_train_loss: 0.1896  avg_val_loss: 0.1172  time: 1528s


EVAL: [60/62] Elapsed 2m 24s (remain 0m 2s) Loss: 0.1088(0.1170) 
EVAL: [61/62] Elapsed 2m 24s (remain 0m 0s) Loss: 0.2141(0.1172) 


...

Epoch 3 - avg_train_loss: 0.1003  avg_val_loss: 0.1058  time: 1558s
INFO:__main__:Epoch 3 - avg_train_loss: 0.1003  avg_val_loss: 0.1058  time: 1558s
Epoch 3 - Score: 0.4611  Scores: [0.48899757604092525, 0.44634853246657347, 0.4298879939098373, 0.46589861382445547, 0.4891564027272441, 0.44619350586786166]
INFO:__main__:Epoch 3 - Score: 0.4611  Scores: [0.48899757604092525, 0.44634853246657347, 0.4298879939098373, 0.46589861382445547, 0.4891564027272441, 0.44619350586786166]
Epoch 3 - Save Best Score: 0.4611 Model
INFO:__main__:Epoch 3 - Save Best Score: 0.4611 Model


Epoch: [4][0/366] Elapsed 0m 6s (remain 41m 46s) Loss: 0.1305(0.1305) Grad: 262257.3125 LR: 0.00001731 
Epoch: [4][20/366] Elapsed 1m 20s (remain 21m 59s) Loss: 0.0821(0.0993) Grad: 93087.1953 LR: 0.00001650 
Epoch: [4][40/366] Elapsed 2m 31s (remain 20m 4s) Loss: 0.1342(0.0960) Grad: 114302.0078 LR: 0.00001570 
Epoch: [4][60/366] Elapsed 3m 52s (remain 19m 23s) Loss: 0.1120(0.0932) Grad: 150924.1719 LR: 0.00001491 
Epoch: [4][80/366] Elapsed 5m 15s (remain 18m 28s) Loss: 0.0942(0.0934) Grad: 107176.5391 LR: 0.00001413 
Epoch: [4][100/366] Elapsed 6m 32s (remain 17m 10s) Loss: 0.1001(0.0930) Grad: 135739.4062 LR: 0.00001336 
Epoch: [4][120/366] Elapsed 7m 46s (remain 15m 45s) Loss: 0.1105(0.0947) Grad: 91071.9062 LR: 0.00001261 
Epoch: [4][140/366] Elapsed 9m 10s (remain 14m 38s) Loss: 0.1040(0.0963) Grad: 234212.8594 LR: 0.00001187 
Epoch: [4][160/366] Elapsed 10m 35s (remain 13m 29s) Loss: 0.1119(0.0971) Grad: 82647.1641 LR: 0.00001115 
Epoch: [4][180/366] Elapsed 11m 59s (remain 12m

Epoch 4 - avg_train_loss: 0.0927  avg_val_loss: 0.1068  time: 1557s
INFO:__main__:Epoch 4 - avg_train_loss: 0.0927  avg_val_loss: 0.1068  time: 1557s
Epoch 4 - Score: 0.4632  Scores: [0.5041783525542941, 0.44718976734588206, 0.431512110805992, 0.46386026167674177, 0.46682214719534976, 0.46565929125048683]
INFO:__main__:Epoch 4 - Score: 0.4632  Scores: [0.5041783525542941, 0.44718976734588206, 0.431512110805992, 0.46386026167674177, 0.46682214719534976, 0.46565929125048683]


Epoch: [5][0/366] Elapsed 0m 2s (remain 14m 31s) Loss: 0.1491(0.1491) Grad: 312498.0625 LR: 0.00000481 
Epoch: [5][20/366] Elapsed 1m 12s (remain 19m 43s) Loss: 0.0414(0.0898) Grad: 121221.5859 LR: 0.00000432 
Epoch: [5][40/366] Elapsed 2m 36s (remain 20m 40s) Loss: 0.0539(0.0850) Grad: 237305.8281 LR: 0.00000385 
Epoch: [5][60/366] Elapsed 4m 3s (remain 20m 16s) Loss: 0.1020(0.0823) Grad: 145323.7500 LR: 0.00000340 
Epoch: [5][80/366] Elapsed 5m 30s (remain 19m 21s) Loss: 0.0825(0.0816) Grad: 281929.4688 LR: 0.00000298 
Epoch: [5][100/366] Elapsed 6m 49s (remain 17m 53s) Loss: 0.1350(0.0825) Grad: 188197.8750 LR: 0.00000259 
Epoch: [5][120/366] Elapsed 8m 14s (remain 16m 40s) Loss: 0.0761(0.0837) Grad: 222539.1250 LR: 0.00000222 
Epoch: [5][140/366] Elapsed 9m 38s (remain 15m 22s) Loss: 0.0900(0.0835) Grad: 118217.5078 LR: 0.00000188 
Epoch: [5][160/366] Elapsed 10m 46s (remain 13m 42s) Loss: 0.0735(0.0844) Grad: 85036.7109 LR: 0.00000157 
Epoch: [5][180/366] Elapsed 11m 51s (remain 1

Epoch 5 - avg_train_loss: 0.0819  avg_val_loss: 0.1003  time: 1536s
INFO:__main__:Epoch 5 - avg_train_loss: 0.0819  avg_val_loss: 0.1003  time: 1536s
Epoch 5 - Score: 0.4485  Scores: [0.4878341040270054, 0.4440003886070633, 0.4134837642686337, 0.44589989488905724, 0.4679583320867184, 0.4318803907390444]
INFO:__main__:Epoch 5 - Score: 0.4485  Scores: [0.4878341040270054, 0.4440003886070633, 0.4134837642686337, 0.44589989488905724, 0.4679583320867184, 0.4318803907390444]
Epoch 5 - Save Best Score: 0.4485 Model
INFO:__main__:Epoch 5 - Save Best Score: 0.4485 Model
========== fold: 3 result ==========
INFO:__main__:========== fold: 3 result ==========
Score: 0.4485  Scores: [0.4878341040270054, 0.4440003886070633, 0.4134837642686337, 0.44589989488905724, 0.4679583320867184, 0.4318803907390444]
INFO:__main__:Score: 0.4485  Scores: [0.4878341040270054, 0.4440003886070633, 0.4134837642686337, 0.44589989488905724, 0.4679583320867184, 0.4318803907390444]
========== CV ==========
INFO:__main__:=

# 作成したモデルをdatasetにアップロード

In [ ]:
# from pathlib import Path
# from kaggle.api.kaggle_api_extended import KaggleApi

# # %cdで/rootに移動できる。/root/.kaggle にkaggle.jsonを配置しなくてはいけないっぽいから、/root/.kaggleディレクトリ作って、そこにkaggle.jsonを配置して、その後もとの/contentに戻っている
# %cd
# !mkdir .kaggle
# !cp /content/drive/MyDrive/kaggle.json .kaggle/
# %cd /content

# # 環境変数設定
# %env KAGGLE_USERNAME=""

# PRETRAIN_DIR = Path(CFG.OUTPUT_DIR)
# api = KaggleApi()
# api.authenticate()
# # dataset-metadata.jsonを作成
# api.dataset_initialize_cli(folder=PRETRAIN_DIR)

In [ ]:
import datetime

# ID = ‘’

UPLOAD_DIR = CFG.OUTPUT_DIR
# UPLOAD_DIR = "/content/sample_data/"

# EX_NO = CFG.identifier
EX_NO = "fb3-miz-single"

# EX_NO = '<実験番号>'  # 実験番号などを入れる、folderのpathにする
# USERID = ''


def dataset_upload():
    import json
    from kaggle.api.kaggle_api_extended import KaggleApi

    kaggle_username = os.environ["KAGGLE_USERNAME"]
    id = f'{kaggle_username}/{EX_NO}'


    dataset_metadata = {}
    #dataset_metadata['id'] = id
    dataset_metadata['id'] = f'{kaggle_username}/{EX_NO}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = f'{EX_NO}'

    with open(UPLOAD_DIR + 'dataset-metadata.json', 'w') as f:
        json.dump(dataset_metadata, f, indent=4)

    api = KaggleApi()
    api.authenticate()

    # # データセットがない場合
    # なぜかapi.dataset_listで「fb3-miz-single」のdatasetが取得できない。。。
    # if f'{kaggle_username}/{EX_NO}' not in [str(d) for d in api.dataset_list(user=kaggle_username, search=f'"{EX_NO}"')]:
    #     api.dataset_create_new(folder=UPLOAD_DIR,
    #                            convert_to_csv=False,
    #                            dir_mode='skip')
    # # データセットがある場合
    # else:
    #     api.dataset_create_version(folder=UPLOAD_DIR,
    #                                version_notes=f"{ver_title} MCRMSE: {_output_log.MCRMSE[0]:.10f}",
    #                                convert_to_csv=False,
    #                                delete_old_versions=True,
    #                                dir_mode='skip')

    api.dataset_create_version(folder=UPLOAD_DIR,
                                version_notes=f"{ver_title} MMSE: {_output_log.MCRMSE[0]:.10f}",
                                convert_to_csv=False,
                                delete_old_versions=False,
                                dir_mode='skip')

    # api.dataset_create_version(folder=UPLOAD_DIR,
    #                             version_notes=f"test upload",
    #                             convert_to_csv=False,
    #                             delete_old_versions=False,
    #                             dir_mode='skip')

dataset_upload()

Skipping folder: tokenizer; use '--dir-mode' to upload folders
Starting upload for file train.log


100%|██████████| 9.55k/9.55k [00:02<00:00, 4.26kB/s]


Upload successful: train.log (10KB)
Starting upload for file microsoft-deberta-v3-base_fold0_best.pth


100%|██████████| 704M/704M [00:28<00:00, 25.9MB/s]


Upload successful: microsoft-deberta-v3-base_fold0_best.pth (704MB)
Skipping folder: config; use '--dir-mode' to upload folders
Skipping folder: model; use '--dir-mode' to upload folders
Starting upload for file microsoft-deberta-v3-base_fold2_best.pth


...

Upload successful: oof_df.pkl (9MB)
Starting upload for file microsoft-deberta-v3-base_fold3_best.pth


100%|██████████| 704M/704M [00:27<00:00, 26.5MB/s]


Upload successful: microsoft-deberta-v3-base_fold3_best.pth (704MB)
Starting upload for file microsoft-deberta-v3-base_fold1_best.pth


100%|██████████| 704M/704M [00:24<00:00, 30.0MB/s]


Upload successful: microsoft-deberta-v3-base_fold1_best.pth (704MB)


In [ ]:
print("完了しました！！！")

In [ ]:
# def beep():
#   from google.colab import output
#   output.eval_js('new Audio(\
# "https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg")\
# .play()') 

# beep()

In [ ]:
# 全ての実行が終了したら、ランタイムを切断する。（GPU使用時間節約）
from google.colab import runtime
runtime.unassign()